In [1]:
!wget https://dumps.wikimedia.org/zhwiki/20240501/zhwiki-20240501-pages-articles-multistream1.xml-p1p187712.bz2
#使用wget工具從指定的URL下載中文維基百科的一部分數據。

--2024-05-18 08:04:44--  https://dumps.wikimedia.org/zhwiki/20240501/zhwiki-20240501-pages-articles-multistream1.xml-p1p187712.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.71, 2620:0:861:3:208:80:154:71
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233419414 (223M) [application/octet-stream]
Saving to: ‘zhwiki-20240501-pages-articles-multistream1.xml-p1p187712.bz2’

zhwiki-20240501-pag 100%[===================>] 222.61M  4.57MB/s    in 49s     

2024-05-18 08:05:33 (4.53 MB/s) - ‘zhwiki-20240501-pages-articles-multistream1.xml-p1p187712.bz2’ saved [233419414/233419414]



In [2]:
!pip install opencc-python-reimplemented
#安裝opencc-python-reimplemented庫，用於進行中文簡繁體轉換。

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 kB 12.8 MB/s eta 0:00:00


In [3]:
from gensim.corpora import WikiCorpus
#從gensim.corpora模組中匯入WikiCorpus類別，用於處理維基百科 XML 轉儲文件
wiki_corpus = WikiCorpus('zhwiki-20240501-pages-articles-multistream1.xml-p1p187712.bz2', dictionary={})
#創建了一個WikiCorpus類別的實例，用來處理指定的維基百科XML轉儲文件。

In [4]:
wiki_corpus
#顯示wiki_corpus變數的內容。

In [5]:
next(iter(wiki_corpus.get_texts()))[:10]
#從wiki_corpus中提取第一篇維基百科文章的文本，並顯示這篇文章的前10個單詞。


['歐幾里得',
 '西元前三世紀的古希臘數學家',
 '而現在被認為是幾何之父',
 '此畫為拉斐爾的作品',
 '雅典學院',
 '数学',
 '是研究數量',
 '屬於形式科學的一種',
 '數學利用抽象化和邏輯推理',
 '從計數']

In [6]:
text_num = 0
#初始化一個計數器text_num，用於記錄處理的文章數量

with open('wiki_text.txt', 'w', encoding='utf-8') as f:
    for text in wiki_corpus.get_texts():
        f.write(' '.join(text)+'\n')
        text_num += 1
        if text_num % 10000 == 0:
            print('{} articles processed.'.format(text_num))

    print('{} articles processed.'.format(text_num))
"""
從wiki_corpus中提取每篇文章的文本，
將其寫入wiki_text.txt文件中，
同時記錄並打印已處理的文章數量，每處理10000篇文章打印一次進度消息。
"""

10000 articles processed.
20000 articles processed.
30000 articles processed.
32786 articles processed.


In [7]:
import jieba
#匯入jieba庫，用於中文將中文文本分割成單詞或短語。
from opencc import OpenCC
#從opencc模組中匯入OpenCC類別，用於中文簡繁轉換。

In [8]:
cc = OpenCC('s2t')
#創建了一個OpenCC類的實例cc，並設置轉換配置為's2t'，將簡體中文轉換為繁體中文。
train_data = open('wiki_text.txt', 'r', encoding='utf-8').read()
#讀取整個文件內容，將其存儲在變數train_data中。
train_data = cc.convert(train_data)
#將文本從簡體中文轉換為繁體中文，並存儲回train_data。
train_data = jieba.lcut(train_data)
#將文本切分為單詞列表，並將結果存儲在train_data中。
train_data = [word for word in train_data if word != '']
#使用列表推導式過濾掉train_data中的空字符串，保留非空的單詞。
train_data = ' '.join(train_data)
#將train_data中的單詞列表連接成一個字符串，單詞之間用空格分隔。
open('seg.txt', 'w', encoding='utf-8').write(train_data)
#將train_data中的內容寫入seg.txt中。

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.525 seconds.
DEBUG:jieba:Loading model cost 1.525 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


136623985

In [1]:
from gensim.models import word2vec
#從gensim.models模組中匯入word2vec類，用於訓練詞嵌入模型。

# Settings
seed = 666
sg = 0
window_size = 10
#vector_size = 100
min_count = 1
workers = 8
#epochs = 5
batch_words = 10000

train_data = word2vec.LineSentence('seg.txt')
#讀取seg.txt文件中的訓練數據，並將其轉換為LineSentence對象。
model = word2vec.Word2Vec(
    train_data,
    min_count=min_count,
    #size=vector_size,
    workers=workers,
    #iter=epochs,
    window=window_size,
    sg=sg,
    seed=seed,
    #隨機數生成器的種子。
    batch_words=batch_words
)

model.save('word2vec.model')
#將訓練好的Word2Vec模型保存為word2vec.model文件。

In [2]:
from gensim.models import word2vec
#從gensim.models模組中匯入word2vec類，用於訓練和使用詞嵌入模型。

string = '電腦'
#定義了一個字符串變數string。
model = word2vec.Word2Vec.load('word2vec.model')
#加載之前訓練並保存的Word2Vec模型。
print(string)
#打印變數string的值。

for item in model.wv.most_similar(string):
#用模型的most_similar方法來查找與string最相似的詞語。
    print(item)
    #打印每個相似詞及其相似度分數。

電腦
('計算機', 0.7834277749061584)
('軟體', 0.7683297395706177)
('pda', 0.7601998448371887)
('硬體', 0.7476744055747986)
('遊戲機', 0.7404897809028625)
('手機', 0.7333237528800964)
('家用', 0.718396008014679)
('顯示器', 0.7166241407394409)
('程式', 0.7159078121185303)
('pc', 0.71230149269104)
